# Bibliotecas

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
import faiss

# Problema

Temos textos com breves descrições de animais e metadados mostrando informações de cobertura (pelo, penas, pele, escamas), e queremos encontrar, destes, os felinos.

## Dados Base

Nos dados de exemplo, temos 5 felinos dentre 20 animais distintos.

In [ ]:
info_animais = [
    "Leões são quadrúpedes musculosos com pelagem curta e dourada, têm presas afiadas e caçam em grupos na savana africana.",
    "Tigres são felinos solitários e robustos, quadrúpedes de pelagem listrada e presas poderosas, que habitam florestas tropicais e pântanos.",
    "Guepardos são quadrúpedes esguios com pelagem amarelada pontilhada de manchas, garras semi-retráteis e corpo adaptado à corrida em planícies abertas.",
    "Leopardos são felinos quadrúpedes ágeis com pelagem amarelada estampada por rosetas, arrastam presas para galhos de árvores e têm presas retráteis.",
    "Gatos domésticos são quadrúpedes pequenos, de pelagens variadas e macias, com dentes caninos afiados e comportamento muitas vezes noturno.",
    "Cachorros são quadrúpedes domesticados, exibem pelagem que varia de curta a longa, têm dentes caninos pronunciados e são altamente sociais.",
    "Elefantes são mamíferos quadrúpedes enormes, com pele espessa quase sem pelos, presas de marfim e tromba versátil, movem-se entre terra e poças d'água.",
    "Cavalos são quadrúpedes elegantes de pelagem curta e crina densa, cascos resistentes e dentes molares fortes, usados para montaria e tração.",
    "Girafas são quadrúpedes extremamente altos, com manchas de pelagem que ajudam na camuflagem e longos pescoços, pastam em copas de árvores baixas.",
    "Zebras são quadrúpedes com pelagem listrada em preto e branco, pelagem curta, cascos duros e vivem em bandos nas planícies africanas.",
    "Rinocerontes são quadrúpedes corpulentos com pele grossa e um ou dois chifres de queratina, pastam em savanas e florestas tropicais.",
    "Hipopótamos são quadrúpedes semi-aquáticos de pele lisa e grossa, com presas fortes, passam grande parte do dia submersos em rios e lagos.",
    "Ursos são quadrúpedes robustos, com pelagem densa, garras afiadas e dentes caninos grandes, percorrendo florestas e montanhas.",
    "Crocodilos são répteis quadrúpedes de pele escamosa, mandíbulas poderosas repletas de dentes pontiagudos, emboscam presas próximas à água.",
    "Golfinhos são mamíferos aquáticos de corpo hidrodinâmico, sem pelos, com nadadeiras e dentes cônicos, nadam em grupos altamente sociais.",
    "Baleias-azuis são mamíferos marinhos gigantes, sem pelos, com corpo alongado e nadadeiras largas, filtram krill com suas barbatanas.",
    "Tubarões-brancos são peixes cartilaginosos de pele áspera, corpo fusiforme e fileiras de dentes triangulares afiados, caçam em mar aberto.",
    "Águias são aves de rapina bípedes, com asas largas e penas resistentes, bico curvo forte e garras afiadas, caçam em grandes altitudes.",
    "Pinguins-imperadores são aves aquáticas bípedes, com plumagem densa e oleosa, asas transformadas em nadadeiras e bico resistente, vivem no gelo.",
    "Papagaios são aves tropicais bípedes, com plumagem colorida, bico curvo forte e patas com dedos opositores, imitam sons do ambiente.",
]

In [ ]:
metadados_animais = [
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pele"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pelo"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "pelo"},
    {"cobertura": "escamas"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "pele"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
    {"cobertura": "penas"},
]

# Solução (Busca Semântica Vetorial)

## Carregando Modelos

In [ ]:
# Escolhido por funcionar bem com português e ser pequeno
modelo = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [ ]:
# Modelo pequeno e rápido de encoding cruzado
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

## Construindo Banco Vetorial

### Criando Embeddings

In [ ]:
# Transformando os documentos dos animais em embeddings
embeddings = modelo.encode(info_animais, convert_to_numpy=True)

In [ ]:
# Normalizando os embeddings
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
print(embeddings)  # Cada embedding é um vetor de 384 dimensões

### Criando Índice no FAISS

In [ ]:
dimensoes = embeddings.shape[1]
index = faiss.IndexFlatIP(dimensoes)
index.add(embeddings)

In [ ]:
# Cada item no index é um vetor numérico:
print(index.reconstruct(0))

## Consulta no Banco

### Consulta Inicial

In [ ]:
# O que quero encontrar no banco?
query = "Felinos nativos e seus detalhes"

In [ ]:
# Criando Embedding e Normalizando a Query
query_embedding = modelo.encode([query], convert_to_numpy=True)
query_embedding = query_embedding / np.linalg.norm(
    query_embedding, axis=1, keepdims=True
)

In [ ]:
print(query_embedding)

In [ ]:
# Procurando nos registros do banco qual mais se assemelham à query
k = 10
D, I = index.search(query_embedding, k)  # Procura os top k  # noqa: E741
print(D, I)  # D = semelhança, I = índice

In [ ]:
# Mostrando resultados iniciais
for semelhanca, indice in zip(D[0], I[0]):
    print(f"Item {indice} ({semelhanca:.3f}) - {info_animais[indice]}")

### Filtragem Híbrida
Utilizando os metadados para obter melhores resultados.

In [ ]:
# Filtrando somente aqueles que tem pelos
candidatos = []
for indice, semelhanca in zip(I[0], D[0]):
    metadados = metadados_animais[indice]
    if metadados["cobertura"] == "pelo":
        candidatos.append((indice, semelhanca))

In [ ]:
# Mostrando resultados após filtragem híbrida
for indice, semelhanca in candidatos:
    print(f"Item {indice} ({semelhanca:.3f}) - {info_animais[indice]}")

### Re-rank

In [ ]:
# Criando os pares consulta/info do animal
pares = [[query, info_animais[indice]] for indice, semelhanca in candidatos]

In [ ]:
# Fazendo o cross-encoding
resultados_cross = cross_encoder.predict(pares)

In [ ]:
print(resultados_cross)  # Para cada índice nos candidatos, mostra o score

In [ ]:
# Combinando índices, score do FAISS e score do Cross-Encoder
resultados = []
for (indice, faiss_score), cross_score in zip(candidatos, resultados_cross):
    resultados.append(
        {
            "indice": indice,
            "faiss": faiss_score,
            "cross": cross_score,
            "info": info_animais[indice],
        }
    )

In [ ]:
# Ordenando os resultados
resultados.sort(key=lambda x: x["cross"], reverse=True)  # por cross, decrescente

In [ ]:
# Mostrando o resultado final
print("Top 5 Felinos Após Re-Rank:")
for r in resultados[:5]:
    print(f"Item {r['indice']} ({r['cross']:.3f}) - {r['info']}")

Os top 5 foram exatamente os 5 felinos. :)